# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выберите данные по странам своей группы (совместно): <br>
    3530203_80101: Spain, Portugal, France, Italy, Belgium<br>
    3530203_80102: Germany, Netherlands <br>
    3530903_80301: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_80302: Sweden, Finland, Norway, Denmark, Iceland<br>
    
2. Попытайтесь найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Найдите и объясните хотя бы 5 правил.
3. Выведите эти правила в отдельных ячейках. 
4. Подумайте, как можно было бы использовать полученные правила на практике.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth, fpmax, association_rules

In [2]:
data = pd.read_csv("D:\edu\Университетские работы\8 сем\Интеллектуальный Анализ Данных\Лаб2\lastfm.csv")
print('data.shape =', data.shape)
data.head()

data.shape = (289955, 4)


,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


##### 1. Data preprocessing

In [3]:
df = data[data.country.isin(['Spain', 'Portugal', 'France', 'Italy', 'Belgium'])]
print('df.shape =', df.shape)
df.head()

df.shape = (27214, 4)


,user,artist,sex,country
83,6,lily allen,m,Portugal
84,6,kanye west,m,Portugal
85,6,sigur rós,m,Portugal
86,6,pink floyd,m,Portugal
87,6,stevie wonder,m,Portugal


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27214 entries, 83 to 289762
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     27214 non-null  int64 
 1   artist   27214 non-null  object
 2   sex      27214 non-null  object
 3   country  27214 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


In [5]:
df[df.duplicated(keep=False)]

,user,artist,sex,country


In [6]:
df.isnull().sum()

user       0
artist     0
sex        0
country    0
dtype: int64

In [7]:
data_by_artists = df.groupby(['user', 'sex', 'country'])['artist'].apply(','.join).reset_index()
data_by_artists_dummies = data_by_artists['artist'].str.get_dummies(',')
data_by_artists_country_dummies = pd.concat([data_by_artists['country'].str.get_dummies(','),
                             data_by_artists['artist'].str.get_dummies(',')], axis = 1)

##### 2. Find frequent itemsets and rules

##### Rule №1

In [8]:
# поиск частых наборов с помощью apriori
frequent_itemsets_apriori = apriori(data_by_artists_dummies, min_support=0.04, use_colnames=True)
frequent_itemsets_apriori

,support,itemsets
0,0.041579,([unknown])
1,0.043693,(a perfect circle)
2,0.073996,(ac/dc)
3,0.092319,(air)
4,0.090204,(amy winehouse)
...,...,...
138,0.047921,"(sigur rós, radiohead)"
139,0.070472,"(the beatles, radiohead)"
140,0.046512,"(radiohead, the cure)"
141,0.040169,"(the killers, radiohead)"


In [9]:
# выделение правил
rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=0.2)
rules_apriori

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(radiohead),(air),0.237491,0.092319,0.048626,0.204748,2.217840,0.026701,1.141376
1,(air),(radiohead),0.092319,0.237491,0.048626,0.526718,2.217840,0.026701,1.611107
2,(arctic monkeys),(coldplay),0.101480,0.188161,0.042988,0.423611,2.251326,0.023893,1.408492
3,(coldplay),(arctic monkeys),0.188161,0.101480,0.042988,0.228464,2.251326,0.023893,1.164587
4,(arctic monkeys),(muse),0.101480,0.174066,0.046512,0.458333,2.633097,0.028847,1.524801
5,(muse),(arctic monkeys),0.174066,0.101480,0.046512,0.267206,2.633097,0.028847,1.226157
6,(arctic monkeys),(radiohead),0.101480,0.237491,0.044397,0.437500,1.842174,0.020297,1.355571
7,(björk),(radiohead),0.074700,0.237491,0.040874,0.547170,2.303958,0.023133,1.683874
8,(coldplay),(franz ferdinand),0.188161,0.095842,0.041579,0.220974,2.305601,0.023545,1.160626
9,(franz ferdinand),(coldplay),0.095842,0.188161,0.041579,0.433824,2.305601,0.023545,1.433898


Значение метрики lift большее 1, указывает, что посылка и следствие чаще встречаются в транзакциях вместе, чем по отдельности. Другими словами, lift больше 1, значит, что если пользователь слушает antecedents, то скорее всего, будет слушать и consequents. Это говорит о степени важности правила.

Значение метрики conviction большее 1 показывает, что ошибки в правиле встречаются реже, чем если бы эти два действия были независимыми. Следствие сильно зависит от посылки.

Таким образом, ниже представлены исполнители, которых можно рекомендовать тем, кто слушает radiohead.

In [10]:
rule1 = rules_apriori[(rules_apriori['antecedents'] == {'radiohead'})].sort_values(by = ['lift'], ascending=False)
rule1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
54,(radiohead),(sigur rós),0.237491,0.086681,0.047921,0.201780,2.327857,0.027335,1.144195
0,(radiohead),(air),0.237491,0.092319,0.048626,0.204748,2.217840,0.026701,1.141376
40,(radiohead),(muse),0.237491,0.174066,0.080338,0.338279,1.943392,0.038999,1.248160
19,(radiohead),(coldplay),0.237491,0.188161,0.083157,0.350148,1.860901,0.038471,1.249269
48,(radiohead),(pink floyd),0.237491,0.130374,0.054968,0.231454,1.775315,0.024006,1.131522
44,(radiohead),(nirvana),0.237491,0.124736,0.051445,0.216617,1.736609,0.021821,1.117288
56,(radiohead),(the beatles),0.237491,0.174066,0.070472,0.296736,1.704730,0.029133,1.174429


##### Rule №2

In [11]:
# поиск частых наборов с помощью FP-growth
frequent_itemsets_fpgrowth = fpgrowth(data_by_artists_dummies, min_support=0.01, use_colnames=True)
frequent_itemsets_fpgrowth

,support,itemsets
0,0.174066,(muse)
1,0.130374,(pink floyd)
2,0.123326,(metallica)
3,0.096547,(depeche mode)
4,0.088090,(u2)
...,...,...
3470,0.016209,"(stereophonics, coldplay)"
3471,0.011980,"(oasis, stereophonics)"
3472,0.011276,"(the killers, stereophonics)"
3473,0.011276,"(stereophonics, coldplay, radiohead)"


Значение метрики lift меньше 1 говорит о том, что пользователь вряд ли будет слушать consequents, если слушает antecedents.

Любитель Metallica вряд ли будет слушать Coldplay (и наоборот тоже), поэтому можно не рекомендовать ему песни этой музыкальной группы.

Также метрика leverage для этих случаев практически равна 0, что говорит о независимости посылок и следствий.

In [12]:
rule2  = association_rules(frequent_itemsets_fpgrowth, metric="confidence", min_threshold=0.1)
rule2 = rule2[rule2.lift < 1]
rule2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
83,(metallica),(radiohead),0.123326,0.237491,0.024665,0.200000,0.842136,-0.004624,0.953136
84,(radiohead),(metallica),0.237491,0.123326,0.024665,0.103858,0.842136,-0.004624,0.978275
85,(metallica),(coldplay),0.123326,0.188161,0.021846,0.177143,0.941445,-0.001359,0.986610
86,(coldplay),(metallica),0.188161,0.123326,0.021846,0.116105,0.941445,-0.001359,0.991830
268,(queen),(radiohead),0.086681,0.237491,0.016913,0.195122,0.821597,-0.003673,0.947359
381,(iron maiden),(the beatles),0.073291,0.174066,0.012685,0.173077,0.994316,-0.000073,0.998804
713,(led zeppelin),(coldplay),0.098661,0.188161,0.017618,0.178571,0.949037,-0.000946,0.988326
845,(bob dylan),(muse),0.074700,0.174066,0.011980,0.160377,0.921358,-0.001023,0.983696
878,(ac/dc),(radiohead),0.073996,0.237491,0.015504,0.209524,0.882238,-0.002069,0.964620
882,(ac/dc),(coldplay),0.073996,0.188161,0.011980,0.161905,0.860460,-0.001943,0.968672


##### Rule №3

In [13]:
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="confidence", min_threshold=0.5)
rules_fpgrowth.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(coldplay, muse)",(radiohead),0.068358,0.237491,0.038760,0.567010,2.387500,0.022525,1.761032
1,"(the beatles, muse)",(radiohead),0.036646,0.237491,0.025370,0.692308,2.915088,0.016667,2.478154
2,"(the beatles, coldplay, muse)",(radiohead),0.016913,0.237491,0.011276,0.666667,2.807122,0.007259,2.287526
3,"(pink floyd, muse)",(radiohead),0.033122,0.237491,0.020437,0.617021,2.598081,0.012571,1.990995
4,"(pink floyd, muse)",(the beatles),0.033122,0.174066,0.016913,0.510638,2.933586,0.011148,1.687778


In [14]:
rules_fpgrowth["antecedent_len"] = rules_fpgrowth["antecedents"].apply(lambda x: len(x))

Высокие значения Lift, Conviction, Confidence говорят о том, что правило сильное, обладает низкой частотностью ошибок и что 56% слушателей massive attack, air также слушают менее популярного исполнителя portishead, т.е. среди слушателей massive attack, air нужно продвигать исполнителя portishead. Все три относятся к стилю трип-хоп.

In [15]:
rule3 = rules_fpgrowth[(rules_fpgrowth['antecedent_len'] > 1) & (rules_fpgrowth['consequents'] == {'portishead'})]
print("rule3.shape =", rule3.shape)
rule3.sort_values(by='lift', ascending=False)

rule3.shape = (1, 10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
172,"(massive attack, air)",(portishead),0.033827,0.08809,0.016913,0.5,5.676,0.013934,1.82382,2


##### Rule №4

In [16]:
frequent_itemsets_fpgrowth_country = fpgrowth(data_by_artists_country_dummies, min_support=0.01, use_colnames=True)
rules_fpgrowth_country = association_rules(frequent_itemsets_fpgrowth_country, metric="lift", min_threshold=0.3)
rules_fpgrowth_country.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Spain),(muse),0.356589,0.174066,0.067653,0.189723,1.089949,0.005583,1.019323
1,(muse),(Spain),0.174066,0.356589,0.067653,0.388664,1.089949,0.005583,1.052467
2,(France),(muse),0.230444,0.174066,0.038760,0.168196,0.966274,-0.001353,0.992942
3,(muse),(France),0.174066,0.230444,0.038760,0.222672,0.966274,-0.001353,0.990002
4,(radiohead),(muse),0.237491,0.174066,0.080338,0.338279,1.943392,0.038999,1.248160


Столбец consequents содержит исполнителей, которые не очень популярны среди слушателей в Испании (метрика lift меньше 1). Поэтому предлагать пополнить ими свои плейлисты и использовать в качестве рекомендаций не стоит.

In [17]:
rules_fpgrowth_country["consequents_len"] = rules_fpgrowth_country["consequents"].apply(lambda x: len(x))
rule4 = rules_fpgrowth_country[((rules_fpgrowth_country['antecedents'] == {'Spain'}))
        & (rules_fpgrowth_country['consequents'].map(lambda x: (('Spain' not in x))))
                        & (rules_fpgrowth_country['consequents_len'] == 1)].sort_values(by='lift', ascending=True)
rule4.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequents_len
77,(Spain),(pink floyd),0.356589,0.130374,0.026075,0.073123,0.560870,-0.020415,0.938232,1
2130,(Spain),(air),0.356589,0.092319,0.019732,0.055336,0.599403,-0.013188,0.960851,1
5404,(Spain),(beirut),0.356589,0.057787,0.012685,0.035573,0.615589,-0.007921,0.976967,1
14346,(Spain),(gorillaz),0.356589,0.050035,0.011276,0.031621,0.631966,-0.006566,0.980984,1
2358,(Spain),(björk),0.356589,0.074700,0.016913,0.047431,0.634947,-0.009724,0.971373,1
14402,(Spain),(sonic youth),0.356589,0.051445,0.012685,0.035573,0.691483,-0.005660,0.983543,1
10276,(Spain),(daft punk),0.356589,0.102185,0.026075,0.073123,0.715592,-0.010363,0.968645,1
5230,(Spain),(the white stripes),0.356589,0.063425,0.016209,0.045455,0.716667,-0.006408,0.981174,1
6823,(Spain),(r.e.m.),0.356589,0.057082,0.014799,0.041502,0.727053,-0.005556,0.983745,1
474,(Spain),(sigur rós),0.356589,0.086681,0.022551,0.063241,0.729586,-0.008358,0.974978,1


##### Rule №5

Слушатели французского исполнителя Manu Chao испанского происхождения, в основном, из Франции и Испании. Бельгия не является его основной аудиторией для получения прибыли.

In [18]:
rule5 = rules_fpgrowth_country[
    ((rules_fpgrowth_country['consequents'] == {'Spain'})
      | (rules_fpgrowth_country['consequents'] == {'Portugal'})
      | (rules_fpgrowth_country['consequents'] == {'France'})
      | (rules_fpgrowth_country['consequents'] == {'Italy'})
      | (rules_fpgrowth_country['consequents'] == {'Belgium'}))
    & (rules_fpgrowth_country['antecedents'] == {'manu chao'})].sort_values(by='lift', ascending=False)
rule5

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequents_len
909,(manu chao),(Italy),0.053559,0.214235,0.012685,0.236842,1.105523,0.001211,1.029623,1
905,(manu chao),(France),0.053559,0.230444,0.012685,0.236842,1.027764,0.000343,1.008384,1
917,(manu chao),(Spain),0.053559,0.356589,0.014799,0.276316,0.774886,-0.004299,0.889077,1


##### Rule №6

Аудитория бельгийской группы Hooverphonic - в Бельгии (lift > 1). Другие страны не являются их аудиторией, выступать там невыгодно.

In [19]:
rule6 = rules_fpgrowth_country[
    ((rules_fpgrowth_country['consequents'] == {'Spain'})
      | (rules_fpgrowth_country['consequents'] == {'Portugal'})
      | (rules_fpgrowth_country['consequents'] == {'France'})
      | (rules_fpgrowth_country['consequents'] == {'Italy'})
      | (rules_fpgrowth_country['consequents'] == {'Belgium'}))
    & (rules_fpgrowth_country['antecedents'] == {'hooverphonic'})].sort_values(by='lift', ascending=False)
rule6

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequents_len
16297,(hooverphonic),(Belgium),0.029598,0.112755,0.014094,0.47619,4.223214,0.010757,1.69383,1
